In [1]:
import json
from gwpy.timeseries import TimeSeries
from gwpy.segments import Segment, SegmentList

from typing import List
from tqdm.auto import tqdm

In [2]:
def interval_intersection(list1: List[List[int]], list2: List[List[int]]) -> List[List[int]]:
    i, j = 0, 0
    intersections = []
    
    while i < len(list1) and j < len(list2):
        a_start, a_end = list1[i]
        b_start, b_end = list2[j]
        
        start = max(a_start, b_start)
        end = min(a_end, b_end)
        
        if start < end: intersections.append([start, end])
        
        if a_end < b_end: i += 1
        else: j += 1
            
    return intersections

In [3]:
with open("l1_o2_segments.json", 'r') as file:
    o2_segments = json.load(file)["segments"]
    no_injection_intval = [
        [1164556817, 1164755435], [1164755459, 1164765305], [1164765357, 1164769204],
        [1164769557, 1178980183], [1178980184, 1187733618]
    ]
    o2_segments = interval_intersection(o2_segments, no_injection_intval)
    
with open("l1_o2_no_cw_inj.json", 'r') as file:
    no_injection_intval = json.load(file)["segments"]
    o2_segments = interval_intersection(o2_segments, no_injection_intval)

duration = sum(segment[1] - segment[0] for segment in o2_segments)
print(f"Total O2 Duration: {duration:,} sec")

Total O2 Duration: 167,583 sec


In [4]:
with open("l1_o3b_segments.json", 'r') as file:
    o3b_segments = json.load(file)["segments"]
    
with open("l1_o3b_no_cw_inj.json", 'r') as file:
    no_injection_intval = json.load(file)["segments"]
    o3b_segments = interval_intersection(o3b_segments, no_injection_intval)

duration = sum(segment[1] - segment[0] for segment in o3b_segments)
print(f"Total O3b Duration: {duration:,} sec")

Total O3b Duration: 1,042,329 sec


In [5]:
gw_event_segments = [
    [1264211182, 1264215278], # GW200128_022011
    [1264314069, 1264318165], # GW200129_065458
    [1264622519, 1264626615], # GW200201_203549
    [1264691364, 1264695460]  # GW200202_154313
]

In [6]:
o2_intervals = [Segment(*segment) for segment in o2_segments]
o3b_intervals = [Segment(*segment) for segment in o3b_segments]

observation_intervals = SegmentList(o2_intervals + o3b_intervals)
gw_event_intervals = SegmentList([Segment(*segment) for segment in gw_event_segments])
observation_intervals = observation_intervals - gw_event_intervals

In [7]:
for interval in observation_intervals:
    print(interval)

[1165564889 ... 1165564904)
[1165566146 ... 1165571297)
[1168788899 ... 1168790410)
[1168813588 ... 1168813600)
[1175713470 ... 1175713574)
[1175713728 ... 1175713874)
[1175714159 ... 1175714174)
[1175714374 ... 1175726426)
[1175730458 ... 1175761375)
[1175763831 ... 1175771194)
[1175773802 ... 1175787563)
[1175789870 ... 1175805747)
[1175809044 ... 1175826548)
[1175828407 ... 1175861674)
[1175864788 ... 1175894543)
[1176035184 ... 1176035317)
[1263588386 ... 1263594319)
[1263597982 ... 1263599374)
[1263601587 ... 1263601875)
[1263612488 ... 1263616279)
[1263618943 ... 1263634588)
[1263637798 ... 1263652909)
[1263674773 ... 1263680830)
[1263680852 ... 1263685120)
[1263688328 ... 1263697638)
[1263697770 ... 1263751884)
[1263752127 ... 1263767422)
[1263777550 ... 1263795012)
[1263806320 ... 1263850547)
[1263855607 ... 1263924944)
[1263942972 ... 1263961795)
[1263966840 ... 1263983837)
[1263986569 ... 1264000088)
[1264001857 ... 1264011283)
[1264013167 ... 1264046619)
[1264046648 ... 1264

In [8]:
duration = sum(interval[1]-interval[0] for interval in observation_intervals)
print(f"Duration: {duration:,} sec")

Duration: 1,193,528 sec


In [9]:
channel_name = "L1"

errors, exists = [], []
for interval in tqdm(observation_intervals, desc="Loading Segments"):
    try:
        data = TimeSeries.fetch_open_data(channel_name, interval[0], interval[1], timeout=300)
        data.write(f"segments/L1-{interval[0]}_{interval[1]}.csv", format="csv")
    except Exception as e:
        errors.append(f"Interval: {interval} not Fetched Due to\nError: {e}")

for error in errors:
    print(error)
    print()

Loading Segments:   0%|          | 0/60 [00:00<?, ?it/s]

KeyboardInterrupt: 